In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)

        # parse the HTML content of the webpage
        soup = BeautifulSoup(response.content, 'html.parser')

        # If a <title> tag exists, soup.title.string fetches its text content. 
        # If the title is missing, it assigns the string "No title found" to self.title.
        self.title = soup.title.string if soup.title else "No title found"

        # This loop iterates over and finds all <script>, <style>, <img>, and <input> elements within the <body> of the HTML.
        # decompose() permanently removes these elements from the soup object to ensure that non-essential content doesn't clutter the text.
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
ed = Website("https://link.springer.com/chapter/10.1007/978-981-15-7219-7_22")
print(ed.title)
print(ed.text)

Convolutional Neural Network Based Chest X-Ray Image Classification for Pneumonia Diagnosis | SpringerLink
Skip to main content
Advertisement
Log in
Menu
Find a journal
Publish with us
Track your research
Search
Cart
Home
Emerging Technology Trends in Electronics, Communication and Networking
Conference paper
Convolutional Neural Network Based Chest X-Ray Image Classification for Pneumonia Diagnosis
Conference paper
First Online:
23 July 2020
pp 254–266
Cite this conference paper
Emerging Technology Trends in Electronics, Communication and Networking
(ET2ECN 2020)
Rushi Bhatt
8
,
Sudhanshusinh Yadav
8
&
Jignesh N. Sarvaiya
8
Part of the book series:
Communications in Computer and Information Science
((CCIS,volume 1214))
Included in the following conference series:
International Conference on Emerging Technology Trends in Electronics Communication and Networking
605
Accesses
7
Citations
Abstract
Pneumonia is one of the most chronic diseases, and therefore, its timely diagnosis is of utm

### A system prompt -- that tells them what task they are performing and what tone they should use

### A user prompt -- the conversation starter that they should reply to

## Messages

The API from OpenAI expects to receive messages in a particular structure.

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [6]:
system_prompt = "You are an assistant that analyzes the contents of a website \
                and provides a short summary, ignoring text that might be navigation related. \
                Respond in markdown."

In [7]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
                    please provide a short summary of this website in markdown. \
                    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [9]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website                 and provides a short summary, ignoring text that might be navigation related.                 Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Convolutional Neural Network Based Chest X-Ray Image Classification for Pneumonia Diagnosis | SpringerLink\nThe contents of this website is as follows;                     please provide a short summary of this website in markdown.                     If it includes news or announcements, then summarize these too.\n\nSkip to main content\nAdvertisement\nLog in\nMenu\nFind a journal\nPublish with us\nTrack your research\nSearch\nCart\nHome\nEmerging Technology Trends in Electronics, Communication and Networking\nConference paper\nConvolutional Neural Network Based Chest X-Ray Image Classification for Pneumonia Diagnosis\nConference paper\nFirst Online:\n23 July 2020\npp 254–266\nCite this confer

In [10]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [11]:
summarize("https://link.springer.com/chapter/10.1007/978-981-15-7219-7_22")

'# Summary of "Convolutional Neural Network Based Chest X-Ray Image Classification for Pneumonia Diagnosis"\n\nThis conference paper, authored by Rushi Bhatt, Sudhanshusinh Yadav, and Jignesh N. Sarvaiya, presents a methodology for diagnosing pneumonia through chest X-ray image classification using Convolutional Neural Networks (CNN). Published in *Emerging Technology Trends in Electronics, Communication and Networking*, it discusses the significance of timely pneumonia diagnosis and the limitations of traditional clinical methods.\n\nThe study describes a proposed CNN model trained on a dataset comprising 4,099 images and tested on 1,757 images, achieving an impressive accuracy of 96.18%. The model was evaluated against a large publicly available dataset named Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification. The results indicate that the proposed model, despite its simpler architecture, outperforms various popular models on multiple performance par

In [12]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [13]:
display_summary("https://link.springer.com/chapter/10.1007/978-981-15-7219-7_22")

# Summary of the Website

The website hosts a conference paper titled **"Convolutional Neural Network Based Chest X-Ray Image Classification for Pneumonia Diagnosis,"** published as part of the *Emerging Technology Trends in Electronics, Communication and Networking* conference proceedings on July 23, 2020. 

## Key Highlights:
- **Authors**: Rushi Bhatt, Sudhanshusinh Yadav, and Jignesh N. Sarvaiya from the Electronics Engineering Department, S. V. National Institute of Technology, Surat, India.
- **Objective**: The study addresses the critical issue of timely pneumonia diagnosis, contrasting traditional clinical methods with more efficient techniques.
- **Methodology**: A Convolutional Neural Network (CNN) approach was developed and trained using a dataset of 4099 chest X-ray images, achieving an impressive accuracy rate of 96.18% when tested on 1757 images.
- **Results**: The CNN model demonstrated better performance compared to several established models, despite its simpler architecture and lack of pre-training.
- **Dataset**: The model was evaluated on the 'Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification' dataset, noted for being one of the largest public datasets available for this task.

This research contributes significantly to the field of medical image classification, particularly in developing automated systems for pneumonia diagnosis via chest X-rays.